In [1]:
import duckdb
import polars
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
con = duckdb.connect('md:')

In [42]:
database = "mlb_uat"

In [43]:
fct_batting_season_totals = con.sql(f"""
    select  
        tot.*,
        players.name,
        games.games as team_g_total

    from {database}.main.fct_batting_season_totals tot
    inner join {database}.main.dim_2023_players players
    on tot.mlbid = players.mlbid
    
    inner join {database}.main.team_games games
    on games.tm_id = players.team
    
    where players.active = 'Y'
""").df()

In [38]:
def run_monte_carlo_sim_event(stat_per_ab, expected_pa, stat_count):
    stat = stat_count.copy()
    stat_tracker = []
    for n in range(int(expected_pa)):
        rand = np.random.random()
        if rand <= stat_per_ab:
            stat += 1
        stat_tracker.append(stat)

    return stat, stat_tracker
        

In [39]:
def iterate(iterations, expected_pa, stat_per_ab, stat_count):
    stat_list = []
    stat_tracker_list = []
    for n in range(iterations):
        stat, stat_tracker = run_monte_carlo_sim_event(stat_per_ab, expected_pa, stat_count)
        stat_list.append(stat)
        stat_tracker_list.append(stat_tracker)
    
    return stat_list, stat_tracker_list


In [40]:
def simulate_season(df):
    # loop over 4040 stats
    ronald_stat_array = {}
    ronald_stat_tracker_list = {}
    for stat in ['hr', 'sb']:
        stat_prob_array = []
        # loop over dataset
        for i in range(len(df)):
            # set/calc vars
            total_games = 162
            total_games_played = fct_batting_season_totals.iloc[i]['g_total']
            stat_count_goal = 40
            stat_count = fct_batting_season_totals.iloc[i][f'{stat}_total']
            stat_count_needed = stat_count_goal - stat_count
            pa = fct_batting_season_totals.iloc[i]['pa_total']
            avg_pa = pa / total_games_played
            # NEED TO RECALCULATE THIS FIELD. TOTAL GAMES PLAYED IS BASED ON IND. GAMES, NOT TEAM GAMES
            expected_pa = avg_pa * (total_games - total_games_played)
            stat_per_ab = stat_count / pa
            iterations = 100
            
            # iterate over monte carlo sim
            stat_list, stat_tracker_list = iterate(iterations, expected_pa, stat_per_ab, stat_count)
            # convert list to array
            stat_array = np.array(stat_list)
            # filter array based on stat count needed
            filter_arr = stat_array >= stat_count_goal
            # append stat probability to list
            stat_prob_array.append(sum(filter_arr)/iterations)
            if df.iloc[i]['mlbid'] == '660670':
                ronald_stat_array[stat] = stat_array.copy()
                ronald_stat_tracker_list[stat] = stat_tracker_list.copy()
        
        # add col for stat count
        df[f'40{stat}prob'] = stat_prob_array
    
    # calculate prob of reaching 40 hr and 40 sb
    df['4040prob'] = df[f'40hrprob'] * df[f'40sbprob']
             
    return df, ronald_stat_array, ronald_stat_tracker_list


In [23]:
df, ronald_stat_array, ronald_stat_tracker_list = simulate_season(fct_batting_season_totals)

/Users/zachshirah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [24]:
df

,mlbid,team,g_total,g_total_rank,pa_total,pa_total_rank,ab_total,ab_total_rank,r_total,r_total_rank,...,obp_total,obp_total_rank,slg_total,slg_total_rank,ops_total,ops_total_rank,team_g_total,40hrprob,40sbprob,4040prob
0,660271,Los Angeles,114.0,4,510.0,6,425.0,29,89.0,5,...,45.475,4,76.642,1,122.118,1,116,1.00,0.0,0.00
1,518692,Los Angeles,113.0,13,519.0,2,453.0,6,101.0,1,...,47.167,1,65.501,2,112.667,2,113,0.02,0.0,0.00
2,660670,Atlanta,112.0,22,516.0,3,449.0,8,101.0,1,...,45.865,2,62.349,5,108.217,3,112,0.28,1.0,0.28
3,605141,Los Angeles,108.0,60,501.0,12,424.0,30,93.0,3,...,40.932,10,64.982,4,105.915,5,113,0.99,0.0,0.00
4,665742,San Diego,115.0,1,504.0,10,400.0,61,65.0,27,...,45.811,3,55.778,11,100.589,6,115,0.03,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,665833,Pittsburgh,9.0,545,40.0,495,32.0,510,7.0,450,...,3.333,504,3.983,500,7.316,495,114,0.00,1.0,0.00
546,573131,Milwaukee,11.0,527,30.0,526,26.0,528,2.0,533,...,3.500,501,2.833,526,6.333,507,114,0.00,0.0,0.00
547,669364,Miami,9.0,545,22.0,552,21.0,544,4.0,489,...,2.667,523,3.000,523,5.667,522,116,0.00,0.0,0.00
548,573131,San Francisco,9.0,545,27.0,532,23.0,536,1.0,562,...,2.784,519,2.533,536,5.317,531,114,0.00,0.0,0.00


In [25]:
ronald_stat_array

{'hr': array([34., 38., 43., 39., 38., 36., 40., 34., 36., 37., 37., 32., 43.,
        35., 37., 35., 34., 37., 37., 33., 42., 34., 35., 42., 35., 40.,
        35., 42., 35., 36., 41., 36., 41., 38., 40., 41., 39., 39., 41.,
        33., 32., 38., 33., 36., 39., 37., 36., 37., 38., 36., 40., 43.,
        39., 41., 40., 38., 44., 37., 36., 38., 38., 39., 38., 41., 35.,
        35., 35., 38., 44., 34., 34., 39., 42., 44., 40., 38., 33., 42.,
        38., 42., 32., 40., 40., 39., 37., 35., 36., 36., 36., 43., 33.,
        32., 35., 36., 35., 38., 34., 45., 35., 34.]),
 'sb': array([82., 73., 81., 83., 71., 74., 75., 72., 76., 80., 73., 69., 70.,
        67., 64., 75., 75., 71., 70., 74., 77., 67., 82., 78., 73., 74.,
        80., 71., 78., 71., 77., 71., 81., 74., 73., 82., 74., 77., 79.,
        78., 71., 80., 78., 79., 75., 70., 66., 77., 71., 76., 68., 84.,
        74., 74., 82., 74., 84., 73., 76., 75., 75., 77., 72., 82., 71.,
        73., 70., 74., 74., 76., 77., 77., 83., 75., 86.,

In [26]:
ronald_stat_tracker_list

{'hr': [[26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   26.0,
   27.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   28.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   29.0,
   2

In [45]:
def create_monte_carlo_4040(df):
    con.sql(f"""
        create or replace table {database}.main.monte_carlo_4040 as
            select *, row_number() over() as row_number from df
    """)
    

In [46]:
create_monte_carlo_4040(df)

In [31]:
def create_acuna_hr_sim_results(stat_tracker_list):
    for stat in stat_tracker_list:
        
        i = 0
        sim_df = pd.DataFrame()
        for stat_tracker in stat_tracker_list[stat]:
            i += 1
            sim_df[f'sim{i}'] = stat_tracker
        
        con.sql(f"create or replace table {database}.main.acuna_{stat}_sim_results as select *, row_number() over() as row_number from sim_df")


In [32]:
create_acuna_hr_sim_results(ronald_stat_tracker_list)

In [33]:
def create_acuna_hr_sim_results_final(stat_array):
    for stat in stat_array:
        sim_results_final = pd.DataFrame(stat_array[stat], columns=['Sim Results'])
        con.sql(f'create or replace table {database}.main.acuna_{stat}_sim_results_final as select * from sim_results_final')


In [35]:
create_acuna_hr_sim_results_final(ronald_stat_array)